In [ ]:
import openai
import json
from datetime import datetime, timedelta
import time
import re
import os

# Configure OpenAI client
client = openai.OpenAI(api_key="Add Key Here")

def clean_json_string(json_str: str) -> str:
    """Clean and validate JSON string"""
    # Remove any markdown code block syntax
    if "```" in json_str:
        pattern = r"```(?:json)?(.*?)```"
        matches = re.findall(pattern, json_str, re.DOTALL)
        if matches:
            json_str = matches[0]
    
    # Remove any leading/trailing whitespace
    json_str = json_str.strip()
    
    # Ensure it starts with [ and ends with ]
    if not json_str.startswith('['):
        json_str = '[' + json_str
    if not json_str.endswith(']'):
        json_str = json_str + ']'
    
    # Replace any invalid escape sequences
    json_str = json_str.replace('\n', ' ').replace('\r', ' ')
    
    return json_str

def generate_data_with_gpt(prompt: str, max_retries: int = 3):
    """Generate data using GPT-4 with retry mechanism"""
    for attempt in range(max_retries):
        try:
            # Add explicit formatting instructions to the prompt
            formatted_prompt = f"""
            Please generate data exactly as specified. Your response must be a valid JSON array.
            Format: {prompt}
            Rules:
            1. Response must be a valid JSON array
            2. All string values must be properly escaped
            3. No comments or explanations, just the JSON array
            4. No markdown formatting
            """
            
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[{
                    "role": "system",
                    "content": "You are a data generation assistant for the Asian Research Network Database. Return only valid JSON arrays without any additional text or formatting."
                }, {
                    "role": "user",
                    "content": formatted_prompt
                }],
                temperature=0.4,  # Lower temperature for more consistent output
                max_tokens=16384  # Maximum tokens for GPT-4
            )
            
            # Get response content
            json_str = response.choices[0].message.content.strip()
            
            # Clean and validate JSON string
            json_str = clean_json_string(json_str)
            
            try:
                # Parse JSON
                data = json.loads(json_str)
                if not isinstance(data, list):
                    raise ValueError("Response is not a JSON array")
                return data
            except json.JSONDecodeError as je:
                print(f"JSON decode error on attempt {attempt + 1}: {str(je)}")
                print(f"Problematic JSON string: {json_str[:200]}...")  # Print first 200 chars for debugging
                if attempt == max_retries - 1:
                    raise
                time.sleep(2)  # Wait longer between retries
                
        except Exception as e:
            if attempt == max_retries - 1:
                raise Exception(f"Failed to generate data after {max_retries} attempts: {str(e)}")
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            time.sleep(2)  # Wait longer between retries

def generate_data_in_batches(prompt_template: str, total_count: int, batch_size: int = 50) -> list:
    """Generate data in smaller batches to avoid token limits"""
    all_data = []
    remaining = total_count
    
    while remaining > 0:
        current_batch = min(batch_size, remaining)
        batch_prompt = prompt_template.replace("<count>", str(current_batch))
        
        try:
            batch_data = generate_data_with_gpt(batch_prompt)
            all_data.extend(batch_data)
            remaining -= current_batch
            print(f"Generated {current_batch} records. {remaining} remaining...")
            time.sleep(1)  # Add small delay between batches
        except Exception as e:
            print(f"Error generating batch: {str(e)}")
            print("Reducing batch size and retrying...")
            batch_size = max(5, batch_size // 2)
            if batch_size < 5:
                raise Exception("Batch size too small, aborting")
    
    return all_data

def generate_insert_statements(table_name: str, data: list) -> str:
    """Generate SQL INSERT statements for the given data"""
    if not data:
        return ""
    
    # Get column names from first item
    columns = list(data[0].keys())
    
    # Start building the SQL string
    sql = f"-- Inserting data into {table_name}\n"
    
    # Generate insert statements in batches
    batch_size = 1000  # Adjust this based on your needs
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        values_list = []
        
        for item in batch:
            values = []
            for col in columns:
                val = item[col]
                if val is None:
                    values.append('NULL')
                elif isinstance(val, (int, float)):
                    values.append(str(val))
                else:
                    values.append(f"'{str(val)}'")
            values_list.append(f"({', '.join(values)})")
        
        sql += f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES\n"
        sql += ',\n'.join(values_list) + ';\n\n'
    
    return sql

def generate_table_data(table_name: str, prompt: str, count: int = None, batch_size: int = None):
    """Generate data for a single table and save to a file"""
    output_dir = 'generated_data'
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f'{table_name.lower()}.sql')
    
    print(f"\nGenerating data for {table_name}...")
    try:
        if count and batch_size:
            data = generate_data_in_batches(prompt, count, batch_size)
        else:
            data = generate_data_with_gpt(prompt)
        
        sql = generate_insert_statements(table_name, data)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(sql)
        
        print(f"Successfully generated data for {table_name}")
        return True
    except Exception as e:
        print(f"Error generating data for {table_name}: {str(e)}")
        return False

def main():
    """Main function to generate all mock data"""
    # Define table generation configurations
    tables = [
        # {
        #     "name": "Country",
        #     "prompt": """Generate 20 countries with their details in this format:
        #     [{"CountryID": <auto_increment>, "CountryName": "<name>", "CountryCode": "<2-letter-code>", "CountryCodeInt": <numeric-code>}]"""
        # },
        # {
        #     "name": "Institution",
        #     "prompt": """Generate 50 institutions in Asia with their details in this format:
        #     [{"InstitutionID": <auto_increment>, "Name": "<name>", "Type": "<university|research_institute|corporate>", 
        #     "Address": "<full_address>", "CountryID": "<1-20>", "Ranking": "<1-500>", "ResearchBudget": "<1000000-100000000>",
        #     "DepartmentCount": "<5-50>", "FoundedYear": "<1800-2000>", "Website": "<url>"}]"""
        # },
        # {
        #     "name": "CollaborationNetwork",
        #     "prompt": """Generate collaboration network data for EXACTLY the same number of institutions as above. Each institution must have exactly one record:
        #     [{"InstitutionID": <match_institution_id>, "PartnerCount": "<5-50>", 
        #     "JointPublications": "<10-200>", "SuccessRate": "<0.3-0.9>"}]
        #     Note: InstitutionID must exactly match an existing institution's ID to maintain one-to-one relationship."""
        # },
        # {
        #     "name": "Department",
        #     "prompt": """Generate <count> departments with their details in this format:
        #     [{"DepartmentID": "DEPT_<unique_id>", "Name": "<name>", "Head": "<full_name>", 
        #     "Budget": "<100000-5000000>", "ResearchFocus": "<focus_area>", "FacultyCount": "<5-50>", 
        #     "InstitutionID": "<1-50>"}]""",
        #     "count": 100,
        #     "batch_size": 25
        # },
        # {
        #     "name": "AcademicStatus",
        #     "prompt": """Generate 10 academic statuses with their details in this format:
        #     [{"AcademicStatusID": <auto_increment>, "AcademicStatus": "<status>"}]"""
        # },
        # {
        #     "name": "Biography",
        #     "prompt": """Generate 100 researcher biographies with their details in this format:
        #     [{"BiographyId": <auto_increment>, "Biography": "<detailed_bio>"}]"""
        # },
        # {
        #     "name": "CurrentPosition",
        #     "prompt": """Generate 100 current positions of the 100 researchers based on the 10 academic statuses with their details in this format:
        #     [{"CurrentPositionID": <auto_increment>, "CurrentPosition": "<position>"}]""",
        # },
        # {
        #     "name": "Researcher",
        #     "prompt": """Generate <count> researchers with their details in this format:
        #     [{"ResearcherID": <auto_increment>, "DepartmentID": "DEPT_<1-100>", "FirstName": "<first>", 
        #     "MiddleName": "<middle>", "LastName": "<last>", "Email": "<email>", "OrcidID": "<orcid>", 
        #     "HIndex": "<0-50>", "TotalCitations": "<0-5000>", "ProfileURL": "<url>", 
        #     "BiographyId": "<1-100>", "CurrentPositionId": "<1-6>", "AcademicStatusId": "<1-10>"}]""",
        #     "count": 100,
        #     "batch_size": 25
        # },
        # {
        #     "name": "ResearchStatement",
        #     "prompt": """Generate 100 research statements with their details in this format:
        #     [{"ResearchStatementID": <auto_increment>, "Statement": "<statement>", "ResearchID": "<1-100>"}]"""
        # },
        # {
        #     "name": "FundingAgency",
        #     "prompt": """Generate 20 funding agencies with their details in this format:
        #     [{"AgencyID": <auto_increment>, "Name": "<name>", 
        #     "Type": "<GOVERNMENT|PRIVATE|ACADEMIC|NON_PROFIT|INTERNATIONAL>", 
        #     "CountryID": "<1-20>", "TotalBudget": "<1000000-100000000>", 
        #     "FocusArea": "<focus_area>", "SuccessRate": "<0.1-0.9>"}]"""
        # },
        # {
        #     "name": "Grants",
        #     "prompt": """Generate <count> grants with their details in this format:
        #     [{"GrantID": "GRANT_<unique_id>", "AgencyID": "AGENCY_<1-20>", "Title": "<title>", 
        #     "Description": "<desc>", "Amount": <50000-5000000>, "CurrencyId": 1, 
        #     "StartDate": "<YYYY-MM-DD HH:mm:ss>", "EndDate": "<YYYY>", 
        #     "GrantStatus": "<DRAFT|SUBMITTED|UNDER_REVIEW|APPROVED|REJECTED|COMPLETED|TERMINATED>",
        #     "FundingAgencyId": "<1-20>", "GrantType": "<type>", "Overhead": <0.1-0.4>}]""",
        #     "count": 200,
        #     "batch_size": 20
        # },
        {
            "name": "Publication",
            "prompt": """Generate <count> publications with their details in this format:
            [{"PublicationID": <auto_increment>, "ConferenceID": "<1-50>", "JournalID": "<1-50>", 
            "Field": "<field>", "TopicID": "TOPIC_<1-50>", "Title": "<title>", "Abstract": "<abstract>", 
            "PublicationDate": "<YYYY-MM-DD>", "Version": "<version>", 
            "Status": "<DRAFT|SUBMITTED|UNDER_REVIEW|ACCEPTED|PUBLISHED|RETRACTED|ARCHIVED>",
            "PageCount": "<10-50>", "Language": "<language>", "LicenseType": "<license>",
            "FullTextURL": "<url>", "CitationCount": <0-1000>}]""",
            "count": 300,
            "batch_size": 100
        }
        #,
        # {
        #     "name": "Review",
        #     "prompt": """Generate <count> reviews with their details in this format:
        #     [{"ReviewID": "REVIEW_<unique_id>", "ResearcherID": "RESEARCHER_<1-100>", 
        #     "PublicationID": "PUB_<1-300>", "Content": "<review_text>", "Score": <1.0-10.0>, 
        #     "Decision": "<ACCEPT|REJECT|MAJOR_REVISION|MINOR_REVISION>", 
        #     "ReviewDate": "<YYYY-MM-DD>", "ReviewerComments": "<comments>", 
        #     "AuthorResponse": "<response>"}]""",
        #     "count": 500,
        #     "batch_size": 50
        # },
        # {
        #     "name": "ResearchTopic",
        #     "prompt": """Generate 50 research topics with their details in this format:
        #     [{"TopicID": "TOPIC_<id>", "Name": "<name>", "Description": "<desc>", 
        #     "ParentTopic": "<null_or_1-50>", "PopularityScore": <0.1-1.0>, "TrendIndex": <-1.0-1.0>}]"""
        # },
        # {
        #     "name": "Dataset",
        #     "prompt": """Generate 100 datasets with their details in this format:
        #     [{"DatasetID": <auto_increment>, "Name": "<name>", "Description": "<desc>", 
        #     "Size": <1000000-100000000000>, "Format": "<format>", "License": "<license>",
        #     "AccessURL": "<url>", "Version": "<version>", "PublicationDate": "<YYYY-MM-DD>"}]"""
        # },
        # {
        #     "name": "DatasetGroup",
        #     "prompt": """Generate 150 dataset group mappings with their details in this format:
        #     [{"DatasetGroupID": <auto_increment>, "PublicationID": <1-300>, "DatasetID": <1-100>}]"""
        # },
        # {
        #     "name": "Journal",
        #     "prompt": """Generate 50 academic journals with their details in this format:
        #     [{"JournalID": <auto_increment>, "Name": "<name>", "ISSN": "<issn>", 
        #     "ImpactFactor": <0.1-15.0>, "Publisher": "<publisher>", "Frequency": "<frequency>",
        #     "Scope": "<scope>", "ReviewTime": <30-180>, "AcceptanceRate": <0.1-0.4>,
        #     "OpenAccessStatus": "<OPEN_ACCESS|SUBSCRIPTION_BASED|HYBRID>"}]"""
        # },
        # {
        #     "name": "Conference",
        #     "prompt": """Generate 50 academic conferences with their details in this format:
        #     [{"ConferenceID": <auto_increment>, "Name": "<name>", "SeriesName": "<series>",
        #     "Date": "<YYYY-MM-DD>", "Location": "<location>", "Ranking": <1-100>,
        #     "AcceptanceRate": <0.1-0.3>, "SubmissionDeadline": "<YYYY-MM-DD>",
        #     "ReviewProcess": "<process>", "VirtualHybridPhysical": "<Virtual|Hybrid|Physical>"}]"""
        # },
        # {
        #     "name": "CitationNetwork",
        #     "prompt": """Generate <count> citation relationships with their details in this format:
        #     [{"CitingPublicationID": <auto_increment>, "CitedPublicationID": <1-300>,
        #     "CitationContext": "<context>", "CitationType": "<methodology|background|comparison|support>",
        #     "CitationImpact": <0.1-1.0>}]""",
        #     "count": 500,
        #     "batch_size": 50
        # },
        # {
        #     "name": "PublicationNetwork",
        #     "prompt": """Generate <count> publication-researcher relationships with their details in this format:
        #     [{"PublicationNetworkID": <auto_increment>, "PublicationD": <1-300>, "ResearcherID": <1-100>}]""",
        #     "count": 400,
        #     "batch_size": 50
        # },
        # {
        #     "name": "ResearchImpact",
        #     "prompt": """Generate <count> research impact records with their details in this format:
        #     [{"ImpactID": "IMPACT_<id>", "Description": "<desc>", "PublicationID": <1-300>,
        #     "Field": "<field>", "CitationScore": <0-100>, "FieldWeightImpact": <0.1-5.0>,
        #     "IndustryImpact": <0.0-1.0>}]""",
        #     "count": 300,
        #     "batch_size": 50
        # },
        # {
        #     "name": "ResearchGroup",
        #     "prompt": """Generate <count> research groups with their details in this format:
        #     [{"GroupID": <auto_increment>, "GrantID": <1-200>, "Name": "<name>", 
        #     "FocusArea": "<focus_area>", "FormationDate": "<YYYY-MM-DD>", 
        #     "Leader": <1-100>, "MemberCount": <5-20>, "ProjectCount": <1-10>}]""",
        #     "count": 50,
        #     "batch_size": 50
        # },
        # {
        #     "name": "ResearchGroupMembers",
        #     "prompt": """Generate <count> research group memberships with their details in this format:
        #     [{"ID": <auto_increment>, "ResearchGroupID": <1-50>, "ResearcherID": <1-100>}]""",
        #     "count": 200,
        #     "batch_size": 50
        # }
    ]
    
    # Generate data for each table
    for table in tables:
        if "count" in table and "batch_size" in table:
            generate_table_data(table["name"], table["prompt"], table["count"], table["batch_size"])
        else:
            generate_table_data(table["name"], table["prompt"])

if __name__ == "__main__":
    main()


Generating data for Country...
Successfully generated data for Country

Generating data for Institution...
Successfully generated data for Institution

Generating data for CollaborationNetwork...
Successfully generated data for CollaborationNetwork

Generating data for Department...
Generated 25 records. 75 remaining...
Generated 25 records. 50 remaining...
Generated 25 records. 25 remaining...
Generated 25 records. 0 remaining...
Successfully generated data for Department

Generating data for AcademicStatus...
Successfully generated data for AcademicStatus

Generating data for Biography...
Successfully generated data for Biography

Generating data for CurrentPosition...
Successfully generated data for CurrentPosition

Generating data for Researcher...
Generated 25 records. 75 remaining...
Generated 25 records. 50 remaining...
Generated 25 records. 25 remaining...
Generated 25 records. 0 remaining...
Successfully generated data for Researcher

Generating data for ResearchStatement...
